## Seq2Seq mapping

In [3]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense

In [4]:
# Load the dataset
df = pd.read_csv('train.csv')

In [5]:
df.head()

,Unnamed: 0,text,gloss
0,0,How are you today?,"TODAY, HOW ARE YOU?"
1,1,What's your favorite food?,"FAVORITE FOOD, WHAT?"
2,2,Where did you go on vacation?,"VACATION WHERE, YOU GO?"
3,3,Did you watch the game last night?,"GAME LAST NIGHT, YOU WATCH?"
4,4,How's work going?,"WORK, HOW GO?"


In [6]:
df.isnull().sum()

Unnamed: 0    0
text          0
gloss         0
dtype: int64

In [7]:
df.describe(include="all")

,Unnamed: 0,text,gloss
count,225.000000,225,225
unique,NaN,225,225
top,NaN,How are you today?,"TODAY, HOW ARE YOU?"
freq,NaN,1,1
mean,73.111111,NaN,NaN
std,52.138304,NaN,NaN
min,0.000000,NaN,NaN
25%,28.000000,NaN,NaN
50%,62.000000,NaN,NaN
75%,118.000000,NaN,NaN


In [8]:
df.shape

(225, 3)

In [9]:
def remove_n(inp):
    return inp.replace("\n", "")

In [10]:
df["gloss"]=df["gloss"].apply(remove_n)

In [11]:
df.head()

,Unnamed: 0,text,gloss
0,0,How are you today?,"TODAY, HOW ARE YOU?"
1,1,What's your favorite food?,"FAVORITE FOOD, WHAT?"
2,2,Where did you go on vacation?,"VACATION WHERE, YOU GO?"
3,3,Did you watch the game last night?,"GAME LAST NIGHT, YOU WATCH?"
4,4,How's work going?,"WORK, HOW GO?"


In [12]:
df["text"]=df["text"].apply(remove_n)

In [13]:
df.head()

,Unnamed: 0,text,gloss
0,0,How are you today?,"TODAY, HOW ARE YOU?"
1,1,What's your favorite food?,"FAVORITE FOOD, WHAT?"
2,2,Where did you go on vacation?,"VACATION WHERE, YOU GO?"
3,3,Did you watch the game last night?,"GAME LAST NIGHT, YOU WATCH?"
4,4,How's work going?,"WORK, HOW GO?"


In [14]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [15]:
text_texts=df["text"].values
gloss_texts=df["gloss"].values
text_texts = ['<start> ' + sentence + ' <end>' for sentence in text_texts]
gloss_texts = ['<start>' + sentence + ' <end>' for sentence in gloss_texts]

In [16]:
text_tokenizer = Tokenizer()
text_tokenizer.fit_on_texts(text_texts)
text_vocab_size = len(text_tokenizer.word_index) + 1
text_sequences = text_tokenizer.texts_to_sequences(text_texts)

In [17]:
gloss_tokenizer = Tokenizer()
gloss_tokenizer.fit_on_texts(gloss_texts)
gloss_vocab_size = len(gloss_tokenizer.word_index) + 1
gloss_sequences = gloss_tokenizer.texts_to_sequences(gloss_texts)

In [18]:
max_text_seq_length = max([len(seq) for seq in text_sequences])
max_gloss_seq_length = max([len(seq) for seq in gloss_sequences])

text_sequences = pad_sequences(text_sequences, maxlen=max_text_seq_length, padding='post')
gloss_sequences = pad_sequences(gloss_sequences, maxlen=max_gloss_seq_length, padding='post')

In [19]:
# Split data into training and validation sets
encoder_input_data, encoder_input_data_val, decoder_input_data, decoder_input_data_val = train_test_split(
    text_sequences, gloss_sequences, test_size=0.2)

# Create decoder target data
decoder_target_data = np.zeros_like(decoder_input_data)
decoder_target_data[:, :-1] = decoder_input_data[:, 1:]
#decoder_target_data[:, -1] = gloss_tokenizer.word_index['<end>']  # Ensure you have an end token

decoder_target_data_val = np.zeros_like(decoder_input_data_val)
decoder_target_data_val[:, :-1] = decoder_input_data_val[:, 1:]
#decoder_target_data_val[:, -1] = gloss_tokenizer.word_index['<end>']  # Ensure you have an end token

In [20]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding

# Define the dimensions
latent_dim = 256

# Define the encoder
encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(text_vocab_size, latent_dim, mask_zero=True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

# Define the decoder
decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(gloss_vocab_size, latent_dim, mask_zero=True)(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(gloss_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the Seq2Seq model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile the model
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')

# Train the model
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=64, epochs=1, validation_data=([encoder_input_data_val, decoder_input_data_val], decoder_target_data_val))





3/3 [==============================] - 15s 2s/step - loss: 6.4985 - val_loss: 6.4828


In [22]:
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding

# Assuming these variables are defined elsewhere in your script
latent_dim = 256
gloss_vocab_size = 5000  # Example size, replace with your actual vocab size

# Define the encoder inputs with the correct shape (timesteps, features)
encoder_inputs = Input(shape=(None, 1))  # Adjust feature dimension as necessary

# Define the encoder LSTM
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
encoder_states = [state_h, state_c]

# Encoder model for inference
encoder_model = Model(encoder_inputs, encoder_states)

# Define the decoder inputs with the correct shape
decoder_inputs = Input(shape=(None, 1))  # Adjust feature dimension as necessary

# Define the decoder LSTM
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_dense = Dense(gloss_vocab_size, activation='softmax')

# Decoder model for inference
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_embedding_inf = Embedding(gloss_vocab_size, latent_dim, mask_zero=True)(decoder_inputs)
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_embedding_inf, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse word index for decoding
reverse_text_word_index = {i: word for word, i in text_tokenizer.word_index.items()}
reverse_gloss_word_index = {i: word for word, i in gloss_tokenizer.word_index.items()}

max_seq_length = 400

# Function to decode the sequence
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1))
    # Populate the first token of target sequence with the start token.
    # target_seq[0, 0] = gloss_tokenizer.word_index['<start>']

    # Sampling loop for a batch of sequences
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_gloss_word_index.get(sampled_token_index, '')
        decoded_sentence += ' ' + sampled_char

        # Exit condition: either hit max length or find stop character.
        if (sampled_char == '<end>' or len(decoded_sentence) > max_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence.strip()

# Example of using the inference function
input_seq = encoder_input_data[1:2]  # Taking the first sequence for translation
translation = decode_sequence(input_seq)
print('Encoded:', translation)


ValueError: Exception encountered when calling layer "lstm_4" (type LSTM).

Dimensions must be equal, but are 1024 and 256 for '{{node mul}} = Mul[T=DT_FLOAT](Sigmoid_1, init_c)' with input shapes: [?,?,1024], [?,256].

Call arguments received by layer "lstm_4" (type LSTM):
  • inputs=['tf.Tensor(shape=(None, None, 1, 256), dtype=float32)', 'tf.Tensor(shape=(None, 256), dtype=float32)', 'tf.Tensor(shape=(None, 256), dtype=float32)']
  • mask=['tf.Tensor(shape=(None, None, 1), dtype=bool)', 'None', 'None']
  • training=None
  • initial_state=None